# 1. Convolution Neural Network
A Convolutional Neural Network (CNN) is a type of deep learning model designed to process data with a grid-like topology, such as images. It is widely used in image classification, object detection, and video analysis.

## Component

### Convolution Layer
Applies filters (kernels) to the input to extract local features like edges, corners, or textures.
```Py
torch.nn.Conv2d(
    in_channels,
    out_channels,
    kernel_size,
    stride=1,
    padding=0,
    ...
)
```
https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html

### Pooling Layer
Reduces the spatial size of the feature maps to lower computational cost and retain important information.

ex) Max Pooling, Average Pooling

```Py
torch.nn.MaxPool2d(
    kernel_size,
    stride=None,
    padding=0
    ...
)
```
https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html

### Relu Activation Function
Introduces non-linearity by setting negative values to zero and keeping positive values.
```Py
torch.nn.functional.relu(
    input
)
```
https://pytorch.org/docs/stable/generated/torch.nn.functional.relu.html

### Dropout
Dropout is a regularization technique used to prevent overfitting in neural networks.
```Py
torch.nn.Dropout(
    p # dropout percent
)
```
https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html


### Fully Connected Layer
Typically at the end of the network, responsible for making final decisions (like classification) based on the extracted features.

```Py
torch.nn.Linear(
    in_features,
    out_features,
    ...
)
```
https://pytorch.org/docs/stable/generated/torch.nn.Linear.html

# 2. Import Libaray

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# 3. MNIST data
The MNIST (Mixed National Institute of Standards and Technology) dataset
is a collection of handwritten digit images, ranging from 0 to 9.

## Detail
Image Size: 28 × 28 pixels, grayscale

Data Split: Training images: 60,000, Test images: 10,000

Number of Classes: 10 (digits from 0 to 9)

<img src="https://storage.googleapis.com/tfds-data/visualization/fig/mnist-3.0.1.png" width="500"/>

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),  # [0,1]
    transforms.Normalize((0.5,), (0.5,))  # [-1,1]
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False)

# 4. Model Implement
<img src="./그림1.jpg" width="500"/>
<img src="./그림2.jpg" width="500"/>


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(64 * 5 * 5, 10)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [ ]:
for epoch in range(15):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss:.4f}")

In [ ]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Accuracy: {accuracy:.4f}")

In [ ]:
from torchsummary import summary
summary(model, input_size=(1, 28, 28))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

classes = [str(i) for i in range(10)]

def imshow(img, title):
    img = img / 2 + 0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.title(title)
    plt.axis('off')

def show_predictions(model, dataloader, device):
    model.eval()
    correct_samples = []
    incorrect_samples = []

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            for img, pred, label in zip(images, predicted, labels):
                if len(correct_samples) < 5 and pred == label:
                    correct_samples.append((img.cpu(), pred.item(), label.item()))
                elif len(incorrect_samples) < 5 and pred != label:
                    incorrect_samples.append((img.cpu(), pred.item(), label.item()))
                
                if len(correct_samples) >= 5 and len(incorrect_samples) >= 5:
                    break
            if len(correct_samples) >= 5 and len(incorrect_samples) >= 5:
                break

    print("✅ correct")
    plt.figure(figsize=(6, 3))
    for i, (img, pred, label) in enumerate(correct_samples):
        plt.subplot(1, 5, i+1)
        imshow(img, f"Pred: {classes[pred]}\nLabel: {classes[label]}")
    plt.tight_layout()
    plt.show()

    print("❌ wrong")
    plt.figure(figsize=(6, 3))
    for i, (img, pred, label) in enumerate(incorrect_samples):
        plt.subplot(1, 5, i+1)
        imshow(img, f"Pred: {classes[pred]}\nLabel: {classes[label]}")
    plt.tight_layout()
    plt.show()

show_predictions(model, test_loader, device)